In [ ]:
from google.colab.patches import cv2_imshow 
from google.colab import drive 
import cv2
import numpy as np
import matplotlib.pyplot as plt

# ImageAnalysis 영상 분석 기법

# K-mean Clustering

K-mean 군집화 구현 1

In [ ]:
#25 X 2 크기의 배열 생성하고 범위안의 랜덤 정수 생성
X = np.random.randint(25,50,(25,2))
Y = np.random.randint(60,85,(25,2))
# 두 배열을 연결
Z = np.vstack((X,Y))

# 배열의 값을 정수에서 실수로 변경 
Z = np.float32(Z)

# 기준을 정의하고 kmeans()함수에 적용
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
ret,label,center=cv2.kmeans(Z,2,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

# 군집을 나누기, 군집 레이블이 0인 데이터만 A 배열에 넣기
A = Z[label.ravel()==0]
# 군집을 나누기, 군집 레이블이 1인 데이터만 B 배열에 넣기
B = Z[label.ravel()==1]

# 군집 화면에 표시하기
plt.scatter(A[:,0],A[:,1], c='b')
plt.scatter(B[:,0],B[:,1],c = 'r')
plt.scatter(center[:,0],center[:,1],s = 200,c = 'y', marker = '*')
plt.xlabel('Height'),plt.ylabel('Weight')
plt.show()

K-mean 군집화 구현 2

In [ ]:
drive.mount('/content/gdrive') 
#영상 불러오기
img = cv2.imread('/content/gdrive/My Drive/Image_Processing/home3.jpg')
#영상의 구조 변경하기
Z = img.reshape((-1,3))
print(Z.shape)

# 배열의 값을 정수에서 실수로 변경 
Z = np.float32(Z)

# 기준을 정의하고 kmeans()함수에 적용
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 5 # 군집 개수
#ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_PP_CENTERS)
ret,label,center=cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)


# center 배열의 값을 uint8 형식으로 되돌리기 (픽셀로 표현해야 함으로), and make original image
center = np.uint8(center)
# label배열 평평(row vector)하게 재조정
res = center[label.flatten()]

#3채널로 다시 조정
res2 = res.reshape((img.shape))
cv2_imshow(res2)

# Region Labrling 영역 레이블링

영역 레이블링 구현

In [ ]:
drive.mount('/content/gdrive') 
#영상 불러오기
img = cv2.imread('/content/gdrive/My Drive/Image_Processing/abcdef.bmp')

# color영상을 gray영상으로 만들기
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# 여기에 이진화 (thresholding)코드가 입력될 수 있음

# 입력 영상에서 흰색 (255) 픽셀 영역을 레이블링 (4 이웃 연산방법을 사용)
num_labels, labels = cv2.connectedComponents(gray_img,4)
print('number of labels:', num_labels)
  
# 각 영역마다 다른 색(레이블)을 갖도록 조정하는 코드
label_H = np.uint8(179*labels/np.max(labels))
blank = 255*np.ones_like(label_H)
labeled_img = cv2.merge([label_H, blank, blank])

# BGR
labeled_img = cv2.cvtColor(labeled_img, cv2.COLOR_HSV2RGB)

# 배경 레이블(0)은 검은색으로
labeled_img[label_H==0] = 0
    
    
# Showing Original Image
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.axis("off")
plt.title("Orginal Image")
plt.show()
    
# 컴포넌트 레이블링된 영상 표현하기
plt.imshow(labeled_img)
plt.axis('off')
plt.title("Component Labeling")
plt.show()

# Contour Tracing 외곽선 추정 기법

외곽선 추정 알고리즘 구하기

In [ ]:
drive.mount('/content/gdrive') 
#영상 불러오기
img = cv2.imread('/content/gdrive/My Drive/Image_Processing/milkdrop.bmp')

# color영상을 gray영상으로 만들기
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#threshold를 이용하여 binary image로 변환
ret, thresh = cv2.threshold(gray_img,127,255,0)

#contours는 point의 list형태.
#hierachy는 contours line의 계층 구조
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
#영상에 외곽선 그리기
image = cv2.drawContours(img,contours, -1, (0,0,255), 1)
print("Segmented image")
cv2_imshow(thresh)
print("Contour tracing image")
cv2_imshow(image)